In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2024-2025] AN2DL/Homework 1

Mounted at /gdrive
/gdrive/My Drive/[2024-2025] AN2DL/Homework 1


In [2]:
%%writefile model.py

import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

def flipHor(images):
    return tf.image.flip_left_right(images)

def flipVer(images):
    return tf.image.flip_up_down(images)

def saturation(images, saturated):
    return tf.image.adjust_saturation(images, saturated)

def centerCrop(images, central_fraction):
    images = tf.image.central_crop(images, central_fraction)
    return images

def shift(images, shift, axis):
    return np.roll(images, shift, axis=axis)

class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """

        self.neural_network = tfk.models.load_model('weights.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """

        preds = self.neural_network.predict(X)

        pred_flip_hor = self.neural_network.predict(flipHor(X))
        pred_flip_ver = self.neural_network.predict(flipVer(X))

        pred_sat = self.neural_network.predict(saturation(X, 4))

        pred_cc = self.neural_network.predict(centerCrop(X, 0.8))

        pred_sh1 = self.neural_network.predict(shift(X, -3, axis=1))
        pred_sh2 = self.neural_network.predict(shift(X, 3, axis=1))

        pred_sh3 = self.neural_network.predict(shift(X, -3, axis=2))
        pred_sh4 = self.neural_network.predict(shift(X, 3, axis=2))


        predictions = np.stack((preds, pred_flip_hor, pred_flip_ver, pred_sat, pred_cc, pred_sh1, pred_sh2, pred_sh3, pred_sh4))
        preds_aggregate = np.mean(predictions, axis=0)

        preds = tf.argmax(preds_aggregate, axis=-1)

        return preds

Overwriting model.py


In [3]:
from datetime import datetime
import zipfile
import os

# Generate a timestamped filename for the zip file
filename = f"submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"

# Create a zip file and add model.py and weights.keras to it
with zipfile.ZipFile(filename, 'w') as zipf:
    zipf.write('model.py')
    zipf.write('weights.keras')

print(f"{filename} has been created successfully in the current directory.")

submission_20241124_143910.zip has been created successfully in the current directory.
